<a href="https://colab.research.google.com/github/Adarsh-hello/-supervised-learning-/blob/main/deepfake%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# !pip install tensorflow opencv-python

import os
import cv2
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:


# ✅ Make folders for extracted data (not for zip files!)
os.makedirs("/content/data/real", exist_ok=True)
os.makedirs("/content/data/fake1", exist_ok=True)

# ✅ Define ZIP file paths (assuming you've uploaded these already)
real_zip_path = "/content/real1.zip"
fake_zip_path = "/content/fake1.zip"
# ✅ Extract real.zip
with zipfile.ZipFile(real_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/data/real")
print("✅ Extracted real.zip to /content/data/real")

# ✅ Extract fake.zip
with zipfile.ZipFile(fake_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/data/fake1")
print("✅ Extracted fake.zip to /content/data/fake1")


In [ ]:
# ✅ STEP 4: Image loading function


def load_images_recursive(real_dir, fake_dir, img_height=128, img_width=128):
    images, labels = [], []
    for label, folder in enumerate([real_dir, fake_dir]):  # 0 = real, 1 = fake
        for root, _, files in os.walk(folder):  # Walk through subdirectories
            for filename in files:
                if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp')):
                    path = os.path.join(root, filename)  # Get full path
                    img = cv2.imread(path)
                    if img is not None:
                        img = cv2.resize(img, (img_width, img_height))
                        img = img / 255.0
                        images.append(img)
                        labels.append(label)
        # Check if any images were loaded for the current label
        if not any(label_val == label for label_val in labels):
            print(f"⚠️ Warning: No images found in {folder} with label {label}")
    # Check if any images were loaded at all
    if not images:
        raise ValueError("❌ No images were loaded. Check your image directories and formats.")
    return np.array(images), np.array(labels)

In [ ]:
import os

print("📂 Real images found:", len(os.listdir("/content/data/real1")))
print("📂 Fake images found:", len(os.listdir("/content/data/fake1")))


In [ ]:
!ls -R /content/data/real
!ls -R /content/data/fake1


In [ ]:
real_dir = "/content/data/real"
fake_dir = "/content/data/fake1"

X, y = load_images_recursive(real_dir, fake_dir)

print(f"✅ X shape: {X.shape}")
print(f"✅ y shape: {y.shape}")


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
y_cat = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)


In [ ]:
# ✅ STEP 6: Simple CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 output: real or fake
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# ✅ STEP 7: Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# 📌 Step 8: Save the Model
model.save("deepfake_detection.h5")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"📊 Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from tensorflow.keras.models import load_model

# ✅ Correct the file path to use the .h5 extension
loaded_model = load_model("deepfake_detection.h5")
print("✅ Model loaded successfully!")


In [ ]:
model.evaluate(X_test, y_test)


In [ ]:
# Step 1: Install required packages
!pip install flask flask-ngrok pyngrok tensorflow keras
!pip install pyngrok

In [ ]:
# Step 2: Setup Flask app + ngrok token
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image
import io

# Step 3: Set your NGROK auth token (replace with your actual token!)
ngrok.set_auth_token("2vd6jnAHywHQhQMsUBi5gKvckBv_7hq1yKFjiiGVsUKvFrH3Z")

# Step 4: Load your trained CNN model
model = tf.keras.models.load_model('deepfake_detection.h5')

# Step 5: Create Flask app
app = Flask(__name__)

# Step 6: HTML Template
html = '''
<!DOCTYPE html>
<html>
<head><title>Deepfake Detection</title></head>
<body style="text-align:center;">
  <h2>Deepfake Detection</h2>
  <form method="POST" enctype="multipart/form-data">
    <input type="file" name="file"><br><br>
    <input type="submit" value="Upload and Predict">
  </form>
  {% if result %}
    <h3>Result: {{ result }}</h3>
  {% endif %}
</body>
</html>
'''

# Step 7: Define route
@app.route('/', methods=['GET', 'POST'])
def upload_predict():
    result = None
    if request.method == 'POST':
        file = request.files['file']
        if file:
            image = Image.open(io.BytesIO(file.read()))
            image = image.resize((128, 128))
            img_array = np.expand_dims(np.array(image)/255.0, axis=0)
            prediction = model.predict(img_array)[0][0]
            result = "FAKE" if prediction > 0.5 else "REAL"
    return render_template_string(html, result=result)

# Step 8: Start server + ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(" * ngrok tunnel link:", public_url)

app.run(port=port)
